# Topic Modeling using Laten Drichlet Allocation on Twitter Accounts

(You can find detailed analysis of all the procedures in the document "cmpe-492-midterm.pdf" in this repository.)

In this notebook, we applied a probabilistic topic modeling algorithm called Latent Drichlet Allocation to detect what kind of topics did a specific user tweet about and what are those topics about.


## Latent Drichlet Allocation (LDA)

Think about a paper which is about using the data analysis to determine the number of genes the organism needs to survive. Assume that by hand, we highlight the words about data analysis in blue, evolutionary biology in pink and genetics in yellow. We see that blue,pink and yellow colors are in different proportions. LDA is a statistical model of document classification that tries to capture above mentioned concept. We define a topic to be a distribution over a dictionary. For instance, genetic topic has genetic related words with high probability and data analysis words with low probability. 

For each document we have, we generate the words in two-stage process:\cite{Blei:2012:PTM:2133806.2133826}
1. Randomly choose a distribution over topics
2. For each word in the document
  * Randomly choose a topic from the distribution over topics in step \#1
  * Randomly choose a word from the corresponding distribution over the vocabulary.
  
We can describe the generative process of LDA formally by the following joint distribution:

$$ p(\beta_{1:K} , \theta_{1:D}, z_{1:D}, w_{1:D}) = \prod_{i=1}^{K} p(\beta_i)   \prod_{d=1}^{D} p(\theta_d) (\prod_{n=1}^{N}  p(z_{d,n}|\theta_d) p(w_{d,n} | \beta_{1:K},z_{d,n}))$$

where $\beta_{1:K}$ are the topics, where each $\beta_k$ is distribution over vocabulary, $\theta_d$ is the topic proportions for document d, where $\theta_{d,k}$ is the topic proportion for topic k in document d, $z_d$ is the topic assignment for document d where $z_{d,n}$ is the topic assignment for the nth word in document d, finally the observed words for document d are $w_d$, where $w_{d,n}$ is the nth word in document d which is an element over a fixed dictionary. We can see that distribution is composed of dependent random variables which define the LDA.

Ref: David M. Blei. Probabilistic topic models. Commun. ACM, 55(4):77–84, April 2012.

## Natural Language Processing (NLP)

The language of twitter is generally close to daily language. People share their ideas and emotions at any time of the day. Other than normal texts, tweets can include hashtags, emoticons, pictures, videos, gifs, urls etc. Even normal text part of the tweets may consist of misspelled words. Apart from these, one user may tweet in lots of language. For example, one tweet may be in Turkish, and another one in English. So we need to make a cleanup before using those tweets. The list of applied processes:

* Remove Twitter Accounts that has less than 2000 words in their tweets
* Remove URLs
* Tokenization
* Stop words
* Remove non-English words from tweets
* Remove non-English accounts
* Delete accounts whose number of left tokens are less than 200
* Stemming
* Remove words that appears only once in the whole corpus

Importing the necessary libraries.

In [1]:
import langid
import logging
import nltk
import numpy as np
import re
import os
import sys
import time
from collections import defaultdict
from string import digits
import pyLDAvis.gensim
import pyLDAvis.sklearn
from gensim import corpora, models, similarities, matutils
import networkx as nx
import string
import math
import pickle

from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.cluster import KMeans

from collections import Counter

C:\Users\Burki\Anaconda3\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Burki\Anaconda3\lib\site-packages\numpy\lib\utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [28]:
def totalWordCount(tList):
    totalWords = 0
    for tt in tList:
        totalWords += len(tt)
    return totalWords

def totalWordCount2(corpus):
    totalWords = 0
    for corp in corpus:
        for c in corp:
            totalWords += c[1]
    return totalWords

#### Read and Remove Twitter Accounts that has less than 2000 words in their tweets

We have already collected tweets of random 900 followers of TRTWorld's twitter account. You can also find those Twitter API codes in this repo.

Here we are reading each user's tweets from files and saving them into a list (tweetList) if the number of words in the file greater than 2000 words.

In [45]:
tweetsList = []
userList = []

for file in os.listdir("tweets3"):
    path = "tweets3\\" + file
    f = open(path, 'r', encoding='utf-8')
    fread = f.read()
    if (len(fread.split()) > 2000):
        tweetsList.append(fread)
        userList.append(file[0:len(file)-4])
    f.close()

print("Number of Users: %d" %(len(tweetsList)))
print("Total Number of Words: %d" %(totalWordCount(tweetsList)))

Number of Users: 1117
Total Number of Words: 328154028


#### Remove URLs

We have removed all urls which are starting with "http://" or "https://. So we excluded all pictures, videos, gifs etc. from the text.

In [ ]:
def remove_urls(text):
    text = re.sub(r"(?:\@|http?\://)\S+", "", text)
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    return text

def doc_rm_urls():
    return [ remove_urls(tweets) for tweets in tweetsList]

tweetsList = doc_rm_urls()

print("Total Number of Words: %d" %(totalWordCount(tweetsList)))

#### Tokenization
Tokenization is basically process of splitting text into words, phrases or other meaningful elements called tokens. We words as our tokens. To better process the text and to create a dictionary and a corpus we tokenized and converted to lower case all the tweets. We used nltk library with regexp to tokenize. 

In [7]:
# This returns a list of tokens / single words for each user
def tokenize_tweet():
    '''
        Tokenizes the raw text of each document
    '''
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    return [ tokenizer.tokenize(t.lower()) for t in tweetsList]

tweetsList = tokenize_tweet()

print("Total Number of Words: " + str(totalWordCount(tweetsList)))

Total Number of Words: 38119510


#### Stop words
Stop words usually refer to the most common words in a language. So being common makes stopwords less effective and sometimes misleading while making decisions. Thus generally stop words are words which are filtered out. We used nltk library to obtain general English stop words, also we determined some words ourselves and also added one and two character words from tweets to stop words.

In [8]:
# Remove stop words
stoplist_tw=['amp','get','got','hey','hmm','hoo','hop','iep','let','ooo','par',
            'pdt','pln','pst','wha','yep','yer','aest','didn','nzdt','via',
            'one','com','new','like','great','make','top','awesome','best',
            'good','wow','yes','say','yay','would','thanks','thank','going',
            'new','use','should','could','best','really','see','want','nice',
            'while','know', 'rt', 'http', 'https']

stoplist  = set(nltk.corpus.stopwords.words("english") + stoplist_tw)

## bu sayi olayini yapicaksak 3d olayina dikkat et
tweetsList = [[token for token in tweets if token not in stoplist and len(token) > 1]
                for tweets in tweetsList]

print("Total Number of Words: " + str(totalWordCount(tweetsList)))

Total Number of Words: 22240226


#### Remove non-English accounts
It is an extension process to removing non-English words. After removing non-English words from tweets, we removed accounts from our corpus whose tweets are majorly not in English. We used a library called langid to detect English accounts.

In [10]:
# Delete Accounts whose tweets are not majorly in English
tweetsList2 = [tweets for tweets in tweetsList if langid.classify(' '.join(tweets))[0] == 'en']

print("Number of Users: " + str(len(tweetsList2)))
print("Total Number of Words: " + str(totalWordCount(tweetsList2)))

Number of Users: 998
Total Number of Words: 19530500


#### Delete accounts whose number of left tokens are less than 200
After all those preprocessing on tweets, we have removed lots of words from original tweets. Some of the accounts, which are possibly not majorly in English but still includes English words, effected more but still existed in the corpus. So to eliminate those misleading accounts from the corpus we deleted accounts whose number of left tokens are less than 200.

In [11]:
# Delete Accounts whose length of tokenized tweets are less than 200
tweetsList2 = [tweets for tweets in tweetsList2 if len(tweets) > 200]
print("Number of Users: " + str(len(tweetsList2)))

Number of Users: 998


#### Remap the User IDs to tweets
In the last two steps we have deleted some accounts. So we need to remap the user ids to the tweets. You can reach tweets from tweetsList2 and users from userList2.

In [12]:
userList2 = []
for i in range(len(tweetsList2)):
    for j in range(i, len(tweetsList)):
        if tweetsList2[i] == tweetsList[j]:
            userList2.append(userList[j])
            break

#### Stemming
For grammatical reasons, documents are going to use different forms of a word, such as organize, organizes, and organizing. Additionally, there are families of derivationally related words with similar meanings, such as democracy, democratic, and democratization. The goal of stemming is to reduce inflectional forms and sometimes derivationally related forms of a word to a common base form. nltk library has mainly 3 kinds of stemming tools for English: lancaster, porter and snowball. We chose Snowball stemmer because it uses a more developed algorithm then Porter Stemmer (Snowball is also called as Porter2) and less aggressive than Lancaster.

In [13]:
# Porter Stemmer and Snowball Stemmer (Porter2) - We useed Snowball Stemmer
# http://stackoverflow.com/questions/10554052/what-are-the-major-differences-and-benefits-of-porter-and-lancaster-stemming-alg

sno = nltk.stem.SnowballStemmer('english')

tweetsList2 = [[sno.stem(token) for token in tweets]
          for tweets in tweetsList2]

print("Total Number of Words: " + str(totalWordCount(tweetsList2)))

Total Number of Words: 19530500


## Word2Vec

In [4]:
wordModel = models.Word2Vec(tweetsList2, size=30, window=5, min_count=11, workers=4)

print(wordModel)

Word2Vec(vocab=45262, size=30, alpha=0.025)


In [21]:
#print(len(wordModel.wv.index2word))
vocab = wordModel.wv.index2word
wordvectors = wordModel.wv[vocab]

In [22]:
kmeansList = np.asarray(wordvectors).astype(np.float64)

kmeans = KMeans(n_clusters=2000).fit(kmeansList)

In [23]:
clusters = {}
labels = {}
centers = []
inVocab = {}

for i in range(0,2000):
    clusters[i] = []

for i, label in enumerate(kmeans.labels_):
    clusters[label].append(vocab[i])
    labels[vocab[i]] = label
    
for c in kmeans.cluster_centers_:
    centers.append(wordModel.similar_by_vector(c)[0][0])
    
for v in vocab:
    inVocab[v] = 1

In [24]:
# Change words in tweets with their cluster center words
tweets2 = [[centers[labels[r]] for r in row if r in inVocab]
          for row in tweetsList2]

### Dictionary and Corpus

To properly use the Twitter data that we have preprocessed, we need to put into a shape that will be understandable by Topic Modeling algorithms. Bag-of-words representation is perfect fit for those kind of algorithms. In bag-of-words we first created a dictionary which consists of all the words from our preprocessed twitter data as values and their ids as keys. Then we created our corpus. Each element of the corpus corresponds to one Twitter account. Each element consists tuples which includes dictionary id of words and the number of that words' occurrences in that account. We used a very useful python library called Gensim to create our dictionary and corpus.

In [25]:
# Build a dictionary where for each document each word has its own id
dictionaryVW = corpora.Dictionary(tweets2)
dictionaryVW.compactify()

# Build the corpus: vectors with occurence of each word for each document
# convert tokenized documents to vectors
corpusVW = [dictionaryVW.doc2bow(tweets) for tweets in tweets2]

print(dictionaryVW)

Dictionary(1993 unique tokens: ['car', 'methan', 'preprint', 'snake', 'tactic']...)


In [26]:
# Normalize word counts by dividing it to the number of elements in its cluster
corpusVW = [[(r[0], int(math.ceil(r[1]/ len(clusters[labels[dictionaryVW[r[0]]]]))) ) for r in row]
          for row in corpusVW]

In [29]:
tweetListVW = []

for c in corpusVW:
    str = ''
    for tokens in c:
        str = str + ((dictionaryVW[tokens[0]]+' ') * tokens[1])
    tweetListVW.append(str)

print("Number of Users: %d"  %(len(tweetListVW)))
print("Total Number of Words: %d" %(totalWordCount2(corpusVW)))

Number of Users: 998
Total Number of Words: 10387073


In [50]:
#clusters

## Training LDA

We used the Python library called Gensim to train our corpus using LDA model. LDA has 3 main parameters need to be optimized. Finding the right parameters for LDA can be considered as an art:

* K, the number of topics
* Alpha, which dictates how many topics a document potentially has. The lower alpha, the lower the number of topics per documents
* Beta, which dictates the number of word per document. Similarly to Alpha, the lower Beta is, the lower the number for words per topic.

Since we are dealing with tweets, we assumed that each follower would have a limited number of topics to tweet about and therefore set alpha to a low value 0.001. (default value is 1.0/num\_topics). We left beta to its default setting. We tried several different values for the number of topics. Too few topics result in heterogeneous set of words while too many diffuse the information with the same words shared across many topics.

In [37]:
t0 = time()

lda_params = {'num_topics': 17, 'passes': 20, 'alpha': 'asymmetric'}

print("Running LDA with: %s  " % lda_params)
lda = models.LdaModel(corpusVW, id2word=dictionaryVW,
                        num_topics=lda_params['num_topics'],
                        passes=lda_params['passes'],
                        alpha = lda_params['alpha'])

print("done in %0.3fs." % (time() - t0))

print()
lda.print_topics()

Running LDA with: {'alpha': 'asymmetric', 'num_topics': 17, 'passes': 20}  
done in 983.846s.



[(0,
  '0.053*"robot" + 0.035*"lego" + 0.032*"omgrobot" + 0.028*"dog" + 0.021*"vex" + 0.019*"fsharp" + 0.017*"mba" + 0.014*"us" + 0.013*"hexbug" + 0.012*"creat"'),
 (1,
  '0.086*"follow" + 0.034*"us" + 0.027*"tweet" + 0.022*"bitcoin" + 0.019*"game" + 0.018*"plm" + 0.017*"bespokebylg" + 0.015*"today" + 0.014*"indiedev" + 0.012*"stori"'),
 (2,
  '0.087*"3dprint" + 0.079*"3d" + 0.045*"printer" + 0.026*"design" + 0.008*"makerspac" + 0.006*"3dprinter" + 0.006*"futur" + 0.006*"first" + 0.006*"help" + 0.006*"free"'),
 (3,
  '0.017*"love" + 0.016*"day" + 0.014*"today" + 0.012*"us" + 0.011*"time" + 0.008*"help" + 0.008*"happi" + 0.007*"year" + 0.007*"look" + 0.007*"live"'),
 (4,
  '0.109*"datasci" + 0.066*"data" + 0.061*"machinelearn" + 0.044*"bigdata" + 0.027*"learn" + 0.021*"scienc" + 0.021*"python" + 0.015*"rstat" + 0.012*"machin" + 0.011*"analyt"'),
 (5,
  '0.086*"robot" + 0.026*"manufactur" + 0.014*"engin" + 0.013*"industri" + 0.011*"autom" + 0.010*"technolog" + 0.009*"product" + 0.008*"uk

In [41]:
# Save Data
dictionaryVW.save('database/word2Vec.dict')
corpora.MmCorpus.serialize('database/word2Vec.mm', corpusVW)
lda.save("database/word2Vec.lda")
with open('database/word2Vec.pickle', 'wb') as f:
    pickle.dump([userList2,tweetListVW] , f)

### Visualization of LDA

The output of the LDA model gives us lots of useful information as expected, word distributions over topics and topic distribution over users. However those information are all hard to read and interpret by looking. Fortunately, we found a library called LDAvis to explore and interpret the results of LDA. LDAvis maps topic similarity by calculating a semantic distance between topics (via Jensen Shannon Divergence)

From this part, you can view all our trials with different parameters and different NLP applications. You can check the change log below to better understand the difference. Top graphic is the latest while bottom one is our first trial.

In [40]:
followers_data =  pyLDAvis.gensim.prepare(lda,corpusVW, dictionaryVW)
pyLDAvis.display(followers_data)

### sklearn NMF-LDA with word2vec

In [90]:
n_samples = len(tweetListVW)
n_features = len(dictionaryVW)
n_topics = 17
n_top_words = 7
n_top_topics = 3

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        sm = sum(topic)
        print("Topic #%d:" % topic_idx)
        for i in topic.argsort()[:-n_top_words - 1:-1]:
            print("(%s, %lf)  " %(feature_names[i], topic[i]/sm), end='')
        print()
    print()
    
def print_top_topics(doc_topic, user, n_top_topics):
    for i in doc_topic[user].argsort()[:-n_top_topics - 1:-1]:
        print("(%d, %lf)  " %(i, doc_topic[user][i]), end='')
    
def topicAndWords(model, doc_topic, user, feature_names):
    model_comp = model.components_
    for i in doc_topic[user].argsort()[:-3 - 1:-1]:
        print("(%d, %lf)  " %(i, doc_topic[user][i]), end='')
        sm = sum(model_comp[i])
        for j in model_comp[i].argsort()[:-3 - 1:-1]:
            print("(%s, %lf)  " %(feature_names[j], model_comp[i][j]/sm), end='')
        print()

In [32]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")

tfidf_vectorizer = TfidfVectorizer(max_features=n_features)
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(tweetListVW)
print("done in %0.3fs." % (time() - t0))

Extracting tf-idf features for NMF...
done in 5.432s.


In [33]:
# 2000 clusters
# Fit the NMF model
print("Fitting the NMF model with tf-idf features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model:")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

#http://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/sklearn.ipynb#topic=0&lambda=1&term=
nmf_vis_data = pyLDAvis.sklearn.prepare(nmf, tfidf, tfidf_vectorizer)
pyLDAvis.display(nmf_vis_data)

Fitting the NMF model with tf-idf features, n_samples=998 and n_features=1993...
done in 3.042s.

Topics in NMF model:
Topic #0:
(time, 0.010074)  (day, 0.008530)  (today, 0.006895)  (year, 0.006844)  (love, 0.006815)  (us, 0.006347)  (thing, 0.005898)  
Topic #1:
(data, 0.201838)  (scienc, 0.034199)  (analyt, 0.031120)  (learn, 0.023497)  (big, 0.023452)  (scientist, 0.020841)  (datasci, 0.017029)  
Topic #2:
(bigdata, 0.230594)  (analyt, 0.110383)  (data, 0.059762)  (big, 0.035160)  (hadoop, 0.020424)  (iiot, 0.018119)  (ai, 0.016931)  
Topic #3:
(learn, 0.046396)  (deep, 0.026438)  (neural, 0.023181)  (paper, 0.022588)  (machin, 0.019998)  (ai, 0.016023)  (model, 0.014811)  
Topic #4:
(3dprint, 0.312139)  (3d, 0.154152)  (printer, 0.081645)  (design, 0.033612)  (3dprinter, 0.031865)  (manufactur, 0.021990)  (filament, 0.012374)  
Topic #5:
(arduino, 0.279140)  (kit, 0.042106)  (raspberrypi, 0.033112)  (makerspac, 0.028972)  (project, 0.027456)  (diy, 0.024432)  (shield, 0.023378)  


In [54]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_features=n_features)
t0 = time()
tf = tf_vectorizer.fit_transform(tweetListVW)
print("done in %0.3fs." % (time() - t0))

Extracting tf features for LDA...
done in 5.494s.


In [55]:
print("Fitting LDA models with tf features, " "n_samples=%d and n_features=%d..." % (n_samples, n_features))
ldaSK = LatentDirichletAllocation(n_topics=n_topics, max_iter=50, learning_method='online', learning_offset=50., random_state=0)
t0 = time()
ldaSK.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(ldaSK, tf_feature_names, n_top_words)

lda_vis_data = pyLDAvis.sklearn.prepare(ldaSK, tf, tf_vectorizer)
pyLDAvis.display(lda_vis_data)

Fitting LDA models with tf features, n_samples=998 and n_features=1993...
done in 220.926s.

Topics in LDA model:
Topic #0:
(data, 0.017690)  (time, 0.012993)  (talk, 0.009912)  (thing, 0.009826)  (post, 0.008344)  (year, 0.007074)  (tweet, 0.006514)  
Topic #1:
(robot, 0.048296)  (omgrobot, 0.040335)  (stem, 0.028657)  (vex, 0.025149)  (forb, 0.018576)  (competit, 0.018150)  (team, 0.017148)  
Topic #2:
(code, 0.030594)  (stem, 0.030189)  (learn, 0.026194)  (edtech, 0.020716)  (teacher, 0.020696)  (school, 0.018025)  (kid, 0.016196)  
Topic #3:
(follow, 0.059180)  (twitter, 0.032586)  (us, 0.030983)  (data, 0.030977)  (tableau, 0.018814)  (dataviz, 0.015606)  (free, 0.012726)  
Topic #4:
(arduino, 0.084807)  (project, 0.025581)  (kit, 0.023225)  (pi, 0.022904)  (robot, 0.022213)  (raspberrypi, 0.019626)  (control, 0.018758)  
Topic #5:
(drone, 0.135757)  (uav, 0.019903)  (launch, 0.010206)  (iphon, 0.007944)  (first, 0.007751)  (ua, 0.007406)  (news, 0.007355)  
Topic #6:
(innov, 0.01

In [63]:
ids = []
chosenNames = ['andrewyng', 'radbuzzz', 'RoboticsEU', 'karpathy', 'polar3d', 'thearduinoguy']

# @andrewyng => 216939636
ids.append(userList2.index('216939636'))
# @radbuzzz => 28953366 
ids.append(userList2.index('28953366'))
# @RoboticsEU => 335419621
ids.append(userList2.index('335419621'))
# @karpathy => 33836629
ids.append(userList2.index('33836629'))
# @polar3d => 2875670213
ids.append(userList2.index('2875670213'))
# @thearduinoguy => 15392736
ids.append(userList2.index('15392736'))

In [69]:
from operator import itemgetter
nmfTopics = nmf.transform(tfidf)
ldaTopics = ldaSK.transform(tf)

In [93]:
for i, id in enumerate(ids):
    print(chosenNames[i])
    print("scikit NMF (v2w)")
    topicAndWords(nmf, nmfTopics, id, tfidf_feature_names)
    print()
    print("scikit LDA (v2w)")
    topicAndWords(ldaSK, ldaTopics, id, tf_feature_names)
    print()
    print("gensim LDA (v2w)")
    print(list(reversed(sorted(lda[corpusVW[id]], key=itemgetter(1))))[:n_top_topics])
    print()

andrewyng
scikit NMF (v2w)
(3, 0.152076)  (learn, 0.046396)  (deep, 0.026438)  (neural, 0.023181)  
(10, 0.073770)  (startup, 0.023130)  (busi, 0.019909)  (innov, 0.018643)  
(0, 0.071584)  (time, 0.010074)  (day, 0.008530)  (today, 0.006895)  

scikit LDA (v2w)
(14, 0.410537)  (learn, 0.041938)  (deep, 0.018938)  (machin, 0.017372)  
(8, 0.203987)  (trump, 0.012357)  (us, 0.011026)  (scienc, 0.009984)  
(15, 0.098413)  (love, 0.016379)  (day, 0.016376)  (us, 0.014419)  

gensim LDA (v2w)
[(15, 0.4823048375650833), (10, 0.21598054637824263), (3, 0.094891426732388387)]

radbuzzz
scikit NMF (v2w)
(4, 0.251557)  (3dprint, 0.312139)  (3d, 0.154152)  (printer, 0.081645)  
(2, 0.044932)  (bigdata, 0.230594)  (analyt, 0.110383)  (data, 0.059762)  
(10, 0.041128)  (startup, 0.023130)  (busi, 0.019909)  (innov, 0.018643)  

scikit LDA (v2w)
(10, 0.302355)  (3dprint, 0.112862)  (3d, 0.104783)  (printer, 0.059287)  
(13, 0.235262)  (data, 0.083728)  (bigdata, 0.062485)  (analyt, 0.052082)  
(8, 0

In [ ]:
gensim LDA

andrewyng
[(15, 0.48172061152763984), (10, 0.21645109819341016), (3, 0.092197346963785592)]
'0.044*"learn" + 0.021*"deep" + 0.020*"machin"
'0.013*"trump" + 0.010*"year" + 0.010*"us" 
'0.017*"love" + 0.016*"day" + 0.014*"today"


radbuzzz
[(2, 0.36363761882489798), (10, 0.22657291528259471), (6, 0.15630377228249343)]
'0.087*"3dprint" + 0.079*"3d" + 0.045*"printer" 
'0.013*"trump" + 0.010*"year" + 0.010*"us" 
0.085*"bigdata" + 0.047*"data" + 0.034*"ai"


RoboticsEU
[(5, 0.71964782964474272), (3, 0.078212371175674852), (10, 0.061950759536093108)]
'0.086*"robot" + 0.026*"manufactur" + 0.014*"engin" 
'0.017*"love" + 0.016*"day" + 0.014*"today"
'0.013*"trump" + 0.010*"year" + 0.010*"us" 

karpathy
[(11, 0.64604950410257989), (15, 0.24672485912868494), (3, 0.05996141157383509)]
'0.013*"time" + 0.010*"thing" + 0.008*"learn" 
'0.044*"learn" + 0.021*"deep" + 0.020*"machin"
'0.017*"love" + 0.016*"day" + 0.014*"today"



polar3d
[(2, 0.62244045071421772), (16, 0.24404842301495563), (3, 0.12745196053857411)]
'0.087*"3dprint" + 0.079*"3d" + 0.045*"printer"
'0.031*"stem" + 0.030*"code" + 0.024*"learn" 
'0.017*"love" + 0.016*"day" + 0.014*"today"


thearduinoguy
[(7, 0.75203259942685219), (3, 0.15457573273012132), (16, 0.03163785260115036)]
'0.051*"arduino" + 0.019*"robot" + 0.017*"project"
'0.017*"love" + 0.016*"day" + 0.014*"today" 
'0.031*"stem" + 0.030*"code" + 0.024*"learn"